In [1]:
import pandas as pd
import numpy as np
import requests
from sklearn.linear_model import LinearRegression
from datetime import datetime

In [2]:
# File paths
menu_file = "menu_data.csv"
busy_times_file = "busy_times.csv"

In [3]:
# OpenWeatherMap API details
api_key = "125893d0d74c6bda2b6a9c9abdd7f8ef"
latitude = 40.76665603  # Replace with actual latitude
longitude = 73.52353817  # Replace with actual longitude
weather_url = f"https://api.openweathermap.org/data/2.5/weather?lat={latitude}&lon={longitude}&appid={api_key}"

# Fetch current weather data
response = requests.get(weather_url)
if response.status_code == 200:
    weather_data = response.json()
    temperature_k = weather_data['main']['temp']  # Temperature in Kelvin
    temperature_f = (temperature_k - 273.15) * 9/5 + 32  # Convert to Fahrenheit
    weather_condition = weather_data['weather'][0]['main']  # e.g., "Rain", "Snow", "Clear"
else:
    print("Failed to fetch weather data")
    temperature_f = 50  # Default to 50°F
    weather_condition = "Clear"

In [4]:
# Load menu prices
menu_df = pd.read_csv(menu_file)  # Assumes columns: "Item", "Base Price"

# Load busy times data
busy_times_df = pd.read_csv(busy_times_file)  # Rows: Days, Columns: Time Slots

In [5]:
# Get current day and time slot
current_day = datetime.now().strftime("%A")  # e.g., "Monday"
current_time = datetime.now().strftime("%I:%M %p")  # e.g., "07:00 PM"

# Match the busy percentage for the current day and time slot
if current_time in busy_times_df.columns:
    busy_percentage = busy_times_df.loc[busy_times_df["Day"] == current_day, current_time].values
    if len(busy_percentage) > 0:
        busy_percentage = busy_percentage[0]
    else:
        busy_percentage = 50  # Default to 50% if no data is found
else:
    busy_percentage = 50  # Default to 50% if the current time slot is missing

In [6]:
def adjust_price(base_price, busy_percentage, temperature_f, weather_condition):
    adjusted_price = base_price
    if temperature_f < 45 or weather_condition in ["Rain", "Snow"] or busy_percentage > 60:
             adjusted_price = base_price * 1.2  # Increase by 20% if conditions are met

    return round(adjusted_price, 2)

menu_df["Temperature (F)"] = temperature_f

# Adjust prices based on conditions
menu_df["Predicted Price"] = menu_df.apply(
    lambda row: adjust_price(
        row["Price"],
        busy_percentage,
        temperature_f,
        weather_condition,
    ),
    axis=1,
)

In [7]:
menu_df

,Restaurant,Address,Opening Hours,Holiday,Name,Price,Temperature (F),Predicted Price
0,Village,"11 W Marie St, Hicksville, NY 11801",Monday to Friday: 11:00 AM - 03:00 PM & 5.00PM...,Tuesday,Rasam,4.95,32.81,5.94
1,Village,"11 W Marie St, Hicksville, NY 11801",Monday to Friday: 11:00 AM - 03:00 PM & 5.00PM...,Tuesday,Sambar,4.95,32.81,5.94
2,Village,"11 W Marie St, Hicksville, NY 11801",Monday to Friday: 11:00 AM - 03:00 PM & 5.00PM...,Tuesday,Mulligatawny Soup,4.95,32.81,5.94
3,Village,"11 W Marie St, Hicksville, NY 11801",Monday to Friday: 11:00 AM - 03:00 PM & 5.00PM...,Tuesday,Garden Soup,4.95,32.81,5.94
4,Village,"11 W Marie St, Hicksville, NY 11801",Monday to Friday: 11:00 AM - 03:00 PM & 5.00PM...,Tuesday,Kachumbar Salad,4.95,32.81,5.94
...,...,...,...,...,...,...,...,...
186,Village,"11 W Marie St, Hicksville, NY 11801",Monday to Friday: 11:00 AM - 03:00 PM & 5.00PM...,Tuesday,Lassi- Chaas,4.95,32.81,5.94
187,Village,"11 W Marie St, Hicksville, NY 11801",Monday to Friday: 11:00 AM - 03:00 PM & 5.00PM...,Tuesday,Hot Drink-Masala Tea,2.95,32.81,3.54
188,Village,"11 W Marie St, Hicksville, NY 11801",Monday to Friday: 11:00 AM - 03:00 PM & 5.00PM...,Tuesday,Hot Drink- Herbal Tea,2.95,32.81,3.54
189,Village,"11 W Marie St, Hicksville, NY 11801",Monday to Friday: 11:00 AM - 03:00 PM & 5.00PM...,Tuesday,Shakes - Mango,4.95,32.81,5.94


In [8]:
# Train a simple regression model
x = menu_df[["Price", "Temperature (F)"]]
y = menu_df["Predicted Price"]

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=40)

In [9]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [23]:
y_pred = model.predict(X_test)
print(y_pred)

[15.53999809  5.94008668  8.34006453 14.34000916  5.94008668  2.3401199
 11.94003131 11.94003131  8.34006453  5.94008668 10.74004238 11.94003131
 15.53999809  7.18807516 11.94003131 14.34000916 14.34000916 14.34000916
 11.94003131 15.53999809 11.94003131 19.13996486  5.94008668  5.94008668
  8.34006453  7.1400756  14.34000916 15.53999809  4.74009775 12.90002245
  2.3401199  11.94003131 13.14002023  7.1400756  14.34000916 14.34000916
 10.74004238  5.94008668 15.53999809]


In [19]:
score = model.score(X_test,y_test)
print(score)

0.9999999942742153
